# Google form analysis

Analysis of results extracted from Google forms in csv format.

## Preparation

In [6]:
%run "../Utilities/Preparation.ipynb"
%run "../Static data/Google form correct answers.ipynb"

C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (18,21,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (14,15,16,17,18,20,21,22,25,26,27,30,33,34,35,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (18,21,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (14,15,16,17,18,20,21,22,25,26,27,30,33,34,35,36,37,

# Constants

In [7]:
gformPath = "../../data/Google forms/"
genericFormName = '2017-11-02-gform-'
enSuffix = 'en'
frSuffix = 'fr'
csvSuffix = '.csv'
gformEN = pd.read_csv(gformPath + genericFormName + enSuffix + csvSuffix)
gformFR = pd.read_csv(gformPath + genericFormName + frSuffix + csvSuffix)

### en-fr gform merge

In [13]:
gformEN['Language'] = pd.Series('en', index=gformEN.index)
gformFR['Language'] = pd.Series('fr', index=gformFR.index)
gformFR.columns = gformEN.columns
gform = pd.concat([gformEN, gformFR])
gform.index = range(0, len(gform.index))

,Timestamp,Are you interested in video games?,Do you play video games?,How old are you?,What is your gender?,How long have you studied biology?,Are you interested in biology?,"Before playing Hero.Coli, had you ever heard about synthetic biology?","Before playing Hero.Coli, had you ever heard about BioBricks?",Have you ever played an older version of Hero.Coli before?,...,What does this device do?.6,What does this device do?.7,"Guess: what would a device producing l-arabinose do, if it started with a l-arabinose-induced promoter?",Guess: the bacterium would glow yellow...,What is the species of the bacterium of the game?,What is the scientific name of the tails of the bacterium?,Find the antibiotic:,You can write down remarks here.,Do not edit - pre-filled anonymous ID,Language
0,2017/08/18 3:22:26 PM GMT+1,Extremely,Extremely,23,Female,Until bachelor's degree,Extremely,Yes,NaN,No,...,It generates antibiotic resistance,It generates green fluorescence in presence of...,"After being induced, it would produce more and...",I don't know,E. Coli,Flagella,Ampicillin,Your game is amazing. One suggestion. In sever...,8d352896-a3f1-471c-8439-0f426df901c1,en
1,2017/09/06 1:28:03 PM GMT+1,Moderately,Moderately,28,Other,Until the end of high school,Moderately,Yes,NaN,No,...,It generates antibiotic resistance,It generates green fluorescence in presence of...,"After being induced, it would produce more and...",I don't know,E. Coli,Flagella,Ampicillin,"This was a nice educational game, thank you fo...",7037c5b2-c286-498e-9784-9a061c778609,en
2,2017/09/12 9:02:06 AM GMT+1,A lot,Moderately,20,Female,Until bachelor's degree,Moderately,No,NaN,No,...,It generates antibiotic resistance,It generates green fluorescence,I don't know,If it produces BFP under purple light,E. Coli,Flagella,Ampicillin,NaN,5c4939b5-425b-4d19-b5d2-0384a515539e,en
3,2017/09/13 2:11:34 AM GMT+1,I don't know,I don't know,0,Other,I don't know,I don't know,No,NaN,No,...,It generates antibiotic resistance,It generates green fluorescence in presence of...,"After being induced, it would produce more and...",If it produces YFP under cyan light,E. Coli,Flagella,Ampicillin,NaN,7825d421-d668-4481-898a-46b51efe40f0,en
4,2017/09/17 6:34:56 PM GMT+1,Moderately,Moderately,21,Male,Until bachelor's degree,A lot,Yes,NaN,No,...,It generates antibiotic resistance,It generates green fluorescence in presence of...,"After being induced, it would produce more and...",If it produces YFP under cyan light,E. Coli,Flagella,Ampicillin,Interested to work on Synthetic Biology as a d...,acb9c989-b4a6-4c4d-81cc-6b5783ec71d8,en
5,2017/10/16 12:46:49 PM GMT+1,Moderately,Rarely,18,Female,Until bachelor's degree,A lot,No,No,No,...,I don't know,I don't know,I don't know,If it produced YFP under cyan light,I don't know,I don't know,Ampicillin,NaN,1a03bc9e-bed4-4ddd-be7f-af23b1d5eb65,en
6,2017/10/16 12:48:34 PM GMT+1,A lot,Rarely,19,Female,Until the end of high school,Extremely,Yes,No,No,...,I don't know,I don't know,I don't know,I don't know,Hero.Coli,Flagella,I don't know,NaN,3685a015-fa97-4457-ad73-da1c50210fe1,en
7,2017/10/16 12:50:38 PM GMT+1,Moderately,Moderately,17,Female,Until the end of middle school,A lot,Yes,No,No,...,I don't know,I don't know,I don't know,I don't know,E. Coli,Flagella,Ampicillin,NaN,9a2e8271-a84d-4b3c-a670-2d65326ed824,en
8,2017/10/16 12:50:40 PM GMT+1,Moderately,Rarely,18,Female,Until bachelor's degree,Extremely,Yes,No,No,...,I don't know,I don't know,"After being induced, it would produce more and...",If it produced YFP under cyan light,Hero.Coli,Flagella,Ampicillin,NaN,bfdfd356-5d6f-4696-a2f1-c1dc338aa64b,en
9,2017/10/16 12:51:49 PM GMT+1,A lot,Moderately,17,Female,Until the end of middle school,Slightly,Yes,No,No,...,I don't know,I don't know,I don't know,I don't know,Hero.Coli,Plasmids,Ampicillin,NaN,a7936587-8b71-43b6-9c61-17b2c2b55de3,en


In [ ]:
gform.head(2)

In [ ]:
gform.tail(2)

In [ ]:
#localplayerguidkey = 'Ne pas modifier - identifiant anonyme prérempli'
localplayerguidkey = 'Do not edit -  pre-filled anonymous ID'
localplayerguidindex = gform.columns.get_loc(localplayerguidkey)
localplayerguidindex

In [ ]:
firstEvaluationQuestionKey = 'In order to modify the abilities of the bacterium, you have to...'
firstEvaluationQuestionIndex = gform.columns.get_loc(firstEvaluationQuestionKey)
firstEvaluationQuestionIndex

In [ ]:
answersColumnNameStem = "answers"
correctionsColumnNameStem = "corrections"

# Functions
<a id=functions />

In [ ]:
#user id in format localplayerguid = '"8d352896-a3f1-471c-8439-0f426df901c1"'

def getAllResponders():
    userIds = gform[localplayerguidkey].values
    result = []
    for id in userIds:
        result.append('"' + id + '"')
    return result

def hasAnswered( userId ):
    return userId.replace('"','') in gform[localplayerguidkey].values

def getAnswers( userId ):
    answers = gform[gform[localplayerguidkey]==userId.replace('"','')]
    _columnAnswers = answers.T
    
    if 0 != len(answers):
        _newColumns = []
        for column in _columnAnswers.columns:
            _newColumns.append(answersColumnNameStem + str(column))
        _columnAnswers.columns = _newColumns
    else:
        # user has never answered
        print("user " + str(userId) + " has never answered")
        
    return _columnAnswers

def getCorrections( userId ):
    _columnAnswers = getAnswers( userId )
    
    if 0 != len(_columnAnswers.columns):

        _questionsCount = len(columnAnswers.values)

        for _columnName in _columnAnswers.columns:
            if answersColumnNameStem in _columnName:
                _answerNumber = _columnName.replace(answersColumnNameStem,"")
                _newCorrectionsColumnName = correctionsColumnNameStem + _answerNumber
                _columnAnswers[_newCorrectionsColumnName] = _columnAnswers[_columnName]

                for _index in range(0, firstEvaluationQuestionIndex):
                    _columnAnswers[_newCorrectionsColumnName][_index] = 'NaN'
                for _index in range(firstEvaluationQuestionIndex, len(_columnAnswers.values)-3):
                    _columnAnswers[_newCorrectionsColumnName][_index] =  \
                        str(columnAnswers[columnName][index]).startswith(str(correctAnswers[columnAnswers[columnName]['Language']][_index]))
                        
                _columnAnswers[_newCorrectionsColumnName][_questionsCount-3] = 'NaN'
                _columnAnswers[_newCorrectionsColumnName][_questionsCount-2] = 'NaN'
                _columnAnswers[_newCorrectionsColumnName][_questionsCount-1] = 'NaN'
        return _columnAnswers
    else:
        # user has never answered
        print("can't give correct answers")
    return _columnAnswers

#function that returns the score from user id
def getScore( userId ):
    score = []
    if hasAnswered( userId ):
        _columnAnswers = getCorrections( userId)
        for _columnName in _columnAnswers.columns:
            # only work on corrected columns
            if correctionsColumnNameStem in _columnName:
                score.append((_columnAnswers[_columnName]).value_counts()[True])
    else:
        print("user " + str(userId) + " has never answered")
    return score

#function that returns the list of checkpoints from user id
def getValidatedCheckpoints( userId ):
    _columnAnswers = getCorrections( userId)
    questionnaireValidatedCheckpointsPerQuestion = pd.Series(np.nan, index=range(len(checkpointQuestionMatching)))
    
    for index in range(0, len(questionnaireValidatedCheckpointsPerQuestion)):
        if _columnAnswers['correct'][index]==True:
            questionnaireValidatedCheckpointsPerQuestion[index] = checkpointQuestionMatching['checkpoint'][index]
        else:
            questionnaireValidatedCheckpointsPerQuestion[index] = ''
    
    questionnaireValidatedCheckpoints = questionnaireValidatedCheckpointsPerQuestion.unique()
    questionnaireValidatedCheckpoints = questionnaireValidatedCheckpoints[questionnaireValidatedCheckpoints!='']
    questionnaireValidatedCheckpoints = pd.Series(questionnaireValidatedCheckpoints)
    questionnaireValidatedCheckpoints = questionnaireValidatedCheckpoints.sort_values()
    questionnaireValidatedCheckpoints.index = range(0, len(questionnaireValidatedCheckpoints))
    return questionnaireValidatedCheckpoints

def getNonValidated( checkpoints ):
    return pd.Series(np.setdiff1d(checkpointQuestionMatching['checkpoint'].values, checkpoints.values))

def getNonValidatedCheckpoints( userId ):
    validated = getValidatedCheckpoints( userId )
    return getNonValidated(validated)